In [1]:
!pip install pandas scikit-learn lazypredict
!pip install pandas scikit-learn shap matplotlib
!pip3 install Biopython
!pip install tensorflow
!pip install keras==2.15.0
!pip install scikeras==0.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cudnn-cu12==8.9.2.26; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requ

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Preprocessing

In [ ]:
df = pd.read_csv("Allergen_Peptide_main.csv")
df['Sequence'] = df['Sequence'].str.replace('\n', '')
#remove space
df['Sequence'] = df['Sequence'].str.replace(' ', '')
# Remove rows with NaN values
df.dropna(inplace=True)
#converting objects to strings
df['Sequence'] = df['Sequence'].astype(str)

In [ ]:
# the set of natural amino acids
natural_amino_acids = set('ACDEFGHIKLMNPQRSTVWY')

# function to check for non-natural amino acids
def has_non_natural(seq):
    non_natural_amino_acids = {'B', 'O', 'J', 'U', 'X', 'Z'}
    return any(aa in non_natural_amino_acids for aa in seq)

# Count and print the number of rows containing non-natural amino acids
#contains_non_natural = df['Sequence'].apply(has_non_natural)
#num_non_natural = contains_non_natural.sum()
#print("Number of rows containing non-natural amino acids:", num_non_natural)#

# Drop rows with non-natural amino acids
rows_to_delete = df[contains_non_natural].index
df.drop(rows_to_delete, inplace=True)

BIoPython features

In [ ]:
import pandas as pd
import Bio
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from google.colab import files

def calculate_aac(sequence):
    analysed_seq = ProteinAnalysis(sequence)
    aac = analysed_seq.get_amino_acids_percent()
    return aac

def calculate_dipeptide_composition(sequence):
    dipeptides = [a+b for a in 'ACDEFGHIKLMNPQRSTVWY' for b in 'ACDEFGHIKLMNPQRSTVWY']
    composition = {dipeptide: 0 for dipeptide in dipeptides}
    for i in range(len(sequence) - 1):
        dipeptide = sequence[i:i+2]
        if dipeptide in composition:
            composition[dipeptide] += 1
    total = sum(composition.values())
    composition = {k: v / total for k, v in composition.items()}
    return composition

def calculate_pseudo_aac(sequence, lamda=10, weight=0.05):
    analysed_seq = ProteinAnalysis(sequence)
    aac = analysed_seq.get_amino_acids_percent()
    theta = [analysed_seq.molecular_weight() / (i + 1) for i in range(lamda)]
    pseudo_aac = {f"PAAC_{i+1}": weight * theta[i] for i in range(lamda)}
    pseudo_aac.update(aac)
    return pseudo_aac

def calculate_physicochemical_properties(sequence):
    analysed_seq = ProteinAnalysis(sequence)
    properties = {
        "Molecular_Weight": analysed_seq.molecular_weight(),
        "Aromaticity": analysed_seq.aromaticity(),
        "Instability_Index": analysed_seq.instability_index(),
        "Isoelectric_Point": analysed_seq.isoelectric_point(),
        "Secondary_Structure_Fraction": analysed_seq.secondary_structure_fraction()
    }
    return properties

def extract_features(input_file):
    output_file = input_file.rstrip('.csv') + "_features.csv"

    df_in = pd.read_csv(input_file)
    sequences = df_in['Sequence'].tolist()

    features = []
    for seq in sequences:
        feature_set = {}
        feature_set.update(calculate_aac(seq))
        feature_set.update(calculate_dipeptide_composition(seq))
        feature_set.update(calculate_pseudo_aac(seq))
        feature_set.update(calculate_physicochemical_properties(seq))

        features.append(feature_set)

    features_df = pd.DataFrame(features)
    result_df = pd.concat([df_in, features_df], axis=1)
    result_df.to_csv(output_file, index=False)

    return result_df, output_file

# Generate features for the input file
df_features, output_file = extract_features('/content/validate_negative_331.csv')

# Display the output DataFrame (optional)
print(df_features.head())

# Download the output features CSV file
files.download(output_file)

In [ ]:
#splitting some attached rows in feature generated file ( specifically for Secondary_structure_fraction feature)
#Secondary_structure_fraction1 = alpha, Secondary_structure_fraction2 = beta, Secondary_structure_fraction 3= coil
# Load the CSV file
csv_file_path = "/content/validate_negative_331_features.csv"
df = pd.read_csv(csv_file_path)

# Split the "Secondary_Structure_Fraction" column into multiple columns
df[['Secondary_Structure_Fraction1', 'Secondary_Structure_Fraction2', 'Secondary_Structure_Fraction3']] = df['Secondary_Structure_Fraction'].str.strip('()').str.split(',', expand=True).astype(float)

# Drop the original "Secondary_Structure_Fraction"
df = df.drop(columns=['Secondary_Structure_Fraction'])

df.head()
df.to_csv('/content/validate_negative_331_features_split.csv', index=False)

ESM features

In [ ]:
pip install fair-esm
import torch
import pandas as pd
from tqdm import tqdm
import esm
pip install git+https://github.com/facebookresearch/esm.git


In [ ]:
import torch
import pandas as pd
import esm
from tqdm import tqdm

# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

# Load the dataset
df = pd.read_csv('train_clusters_final.csv')

# Extract sequences and sequence IDs
sequences = df['Sequence'].tolist()
sequence_ids = df['Sequence_ID'].tolist()

# Prepare data in the required format for ESM
data = list(zip(sequence_ids, sequences))

# Set the batch size
batch_size = 50

# Initialize the list to store sequence representations
sequence_representations = []

# Process data in batches
for start in tqdm(range(0, len(data), batch_size), desc="Processing ESM Batches"):
    end = start + batch_size
    batch_data = data[start:end]

    try:
        # Use batch_converter to get labels, batch tokens, and lengths
        batch_labels, batch_tokens, batch_lengths = batch_converter(batch_data)

        # Ensure batch_tokens is converted to a list of numerical tokens
        batch_tokens = torch.tensor(batch_tokens).to(device)
    except Exception as e:
        print(f"Error in batch_converter: {e}")
        continue

    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33])
        token_embeddings = results["representations"][33]

    # Average the representations for each sequence to get fixed-length embeddings
    for i, (seq_id, seq) in enumerate(batch_data):
        seq_embedding = token_embeddings[i, 1:len(seq) + 1].mean(0).cpu().numpy()
        sequence_representations.append(seq_embedding)

# Assuming you want to add these embeddings to the DataFrame and save it
# Add embeddings to DataFrame
df['esm_embeddings'] = sequence_representations

# Save DataFrame to CSV file
df.to_csv('train_clusters_final_with_esm_embeddings.csv', index=False)


In [ ]:
# Create a DataFrame for the ESM embeddings
esm_embeddings_df = pd.DataFrame(sequence_representations, columns=[f'esm_feature_{i}' for i in range(sequence_representations[0].shape[0])])
esm_embeddings_df['Sequence_ID'] = sequence_ids

# Merge ESM embeddings with the original DataFrame
merged_df = pd.merge(df, esm_embeddings_df, on='Sequence_ID')

# Save the merged DataFrame to a CSV file
merged_df.to_csv('train_clusters_final_esm_embeddings.csv', index=False)

print("ESM embeddings extraction and merging complete.")

BioPython feature Selection

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel

# data loading
train_data = pd.read_csv('/content/train_data_1727.csv')
test_data = pd.read_csv('/content/test_data_1727.csv')
validation_data = pd.read_csv('/content/validation_data_433.csv')

# Separate features and labels
X_train = train_data.drop(columns=["Label"])
y_train = train_data["Label"]
X_test = test_data.drop(columns=["Label"])
y_test = test_data["Label"]
validation_X = validation_data.drop(columns=["Label"])
validation_y = validation_data['Label']

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
validation_X_scaled = scaler.transform(validation_X)

# Feature Engineering - Gradient Boosting Feature Importance
gb = GradientBoostingClassifier(n_estimators=100, random_state=0)
gb.fit(X_train_scaled, y_train)

selector = SelectFromModel(gb, prefit=True, threshold='mean')
X_train_selected = selector.transform(X_train_scaled)
X_test_selected = selector.transform(X_test_scaled)
validation_X_selected = selector.transform(validation_X_scaled)

# Get selected feature names
selected_features = X_train.columns[selector.get_support()]

print("Selected Features (based on Gradient Boosting):")
print(selected_features)

ESM feature selection

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from matplotlib.colors import ListedColormap

# data loading
train_data = pd.read_csv('/content/train_data_esm.csv')
test_data = pd.read_csv('/content/test_data_esm.csv')
validation_data = pd.read_csv('/content/validation_data_433_esm.csv')

# Separate features and labels
X_train = train_data.drop(columns=["Label"])
y_train = train_data["Label"]
X_test = test_data.drop(columns=["Label"])
y_test = test_data["Label"]
validation_X = validation_data.drop(columns=["Label"])
validation_y = validation_data['Label']

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
validation_X_scaled = scaler.transform(validation_X)

# Feature Engineering - Gradient Boosting Feature Importance
gb = GradientBoostingClassifier(n_estimators=100, random_state=0)
gb.fit(X_train_scaled, y_train)

selector = SelectFromModel(gb, prefit=True, threshold='mean')
X_train_selected = selector.transform(X_train_scaled)
X_test_selected = selector.transform(X_test_scaled)
validation_X_selected = selector.transform(validation_X_scaled)

# Get selected feature names
selected_features = X_train.columns[selector.get_support()]

print("Selected Features (based on Gradient Boosting):")
print(selected_features)

# Define a function to plot 3D t-SNE results with improved visualization and save the plot
def plot_tsne_3d(X, y, title, save_path=None):
    tsne = TSNE(n_components=3, random_state=0, perplexity=30, learning_rate=200)
    X_tsne = tsne.fit_transform(X)

    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Create a custom colormap
    cmap = ListedColormap(['#cc3f3f', '#498fc9'])

    scatter = ax.scatter(X_tsne[:, 0], X_tsne[:, 1], X_tsne[:, 2], c=y, cmap=cmap, s=20, alpha=0.7)
    #legend = ax.legend(*scatter.legend_elements(), title="Label")
    #ax.add_artist(legend)

    #ax.set_title(title)

    if save_path:
        plt.savefig(save_path, format='jpeg')

    plt.show()

# Plot and save 3D t-SNE for original features
plot_tsne_3d(X_train_scaled, y_train, '3D t-SNE of All Features_ss_rb_ESM', save_path='3d_tsne_all_features_ss_rb_ESM.jpeg')

# Plot and save 3D t-SNE for selected features
plot_tsne_3d(X_train_selected, y_train, '3D t-SNE of Selected Features_ss_rb_ESM', save_path='3d_tsne_selected_features_ss_rb_ESM.jpeg')


Model (ESM+BIO)

In [1]:
!pip install tensorflow==2.12.0

In [2]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifierCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score, confusion_matrix

# Load datasets
train_data = pd.read_csv('train_data_esm.csv')
test_data = pd.read_csv('test_data_esm.csv')
validation_data = pd.read_csv('validation_data_433_esm.csv')

# Define selected features
selected_features_main = [
    'feature_2', 'feature_3', 'feature_10', 'feature_18', 'feature_26',
    'feature_27', 'feature_38', 'feature_57', 'feature_65', 'feature_68',
    'feature_90', 'feature_91', 'feature_101', 'feature_113', 'feature_119',
    'feature_141', 'feature_149', 'feature_162', 'feature_163', 'feature_171',
    'feature_173', 'feature_186', 'feature_190', 'feature_195', 'feature_197',
    'feature_210', 'feature_222', 'feature_225', 'feature_230', 'feature_234',
    'feature_235', 'feature_254', 'feature_265', 'feature_271', 'feature_276',
    'feature_298', 'feature_313', 'feature_317', 'feature_322', 'feature_333',
    'feature_356', 'feature_365', 'feature_382', 'feature_383', 'feature_396',
    'feature_404', 'feature_414', 'feature_417', 'feature_440', 'feature_482',
    'feature_492', 'feature_497', 'feature_515', 'feature_520', 'feature_526',
    'feature_534', 'feature_546', 'feature_565', 'feature_568', 'feature_615',
    'feature_643', 'feature_652', 'feature_653', 'feature_654', 'feature_657',
    'feature_660', 'feature_690', 'feature_697', 'feature_704', 'feature_717',
    'feature_719', 'feature_744', 'feature_784', 'feature_793', 'feature_800',
    'feature_802', 'feature_805', 'feature_818', 'feature_830', 'feature_841',
    'feature_842', 'feature_843', 'feature_853', 'feature_855', 'feature_858',
    'feature_860', 'feature_869', 'feature_876', 'feature_878', 'feature_880',
    'feature_895', 'feature_896', 'feature_914', 'feature_941', 'feature_943',
    'feature_946', 'feature_979', 'feature_984', 'feature_988', 'feature_989',
    'feature_992', 'feature_1003', 'feature_1007', 'feature_1013', 'feature_1015',
    'feature_1025', 'feature_1031', 'feature_1048', 'feature_1051', 'feature_1065',
    'feature_1069', 'feature_1072', 'feature_1077', 'feature_1085', 'feature_1093',
    'feature_1112', 'feature_1119', 'feature_1122', 'feature_1144', 'feature_1145',
    'feature_1149', 'feature_1158', 'feature_1162', 'feature_1163', 'feature_1182',
    'feature_1185', 'feature_1213', 'feature_1223', 'feature_1228', 'feature_1233',
    'feature_1245', 'feature_1246', 'feature_1248'
]

# Define the selected features for the QDA classifier
selected_features_qda = [
    'C', 'D', 'E', 'K', 'W', 'AA', 'AC', 'AE', 'AH', 'AT', 'CC', 'CG', 'CK',
    'CN', 'DK', 'EC', 'EE', 'EQ', 'FE', 'FT', 'GA', 'GQ', 'GW', 'HF', 'HP',
    'IE', 'IK', 'IN', 'KD', 'LE', 'LS', 'MG', 'ML', 'NW', 'NY', 'PP', 'RL',
    'RV', 'ST', 'SV', 'VT', 'WC', 'WD', 'YF', 'YG', 'YR', 'YY', 'PAAC_5',
    'PAAC_9', 'Aromaticity', 'Isoelectric_Point',
    'Secondary_Structure_Fraction1', 'Secondary_Structure_Fraction3'
]

# Load QDA specific datasets
train_data_qda = pd.read_csv('train_data_1727.csv')
test_data_qda = pd.read_csv('test_data_1727.csv')
validation_data_qda = pd.read_csv('validation_data_433.csv')

# Prepare data for QDA
X_train_qda_selected = train_data_qda[selected_features_qda]
X_test_qda_selected = test_data_qda[selected_features_qda]
validation_X_qda_selected = validation_data_qda[selected_features_qda]

y_train_qda = train_data_qda['Label']
y_test_qda = test_data_qda['Label']
validation_y_qda = validation_data_qda['Label']

# Prepare data for main models
X_train_selected = train_data[selected_features_main]
y_train = train_data['Label']
X_test_selected = test_data[selected_features_main]
y_test = test_data['Label']
validation_X_selected = validation_data[selected_features_main]
validation_y = validation_data['Label']

# Define ANN model
def create_ann_model(input_dim=196, dropout_rate=0.5):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Define base models
base_models = [
    ('svc', Pipeline([('scaler', StandardScaler()), ('svc', SVC(probability=True))])),
    ('qda', Pipeline([('scaler', StandardScaler()), ('qda', QuadraticDiscriminantAnalysis())])),
    ('ann', KerasClassifier(build_fn=create_ann_model, input_dim=X_train_selected.shape[1])),
    ('knn', Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier())]))
]

# Initialize arrays for meta-features
meta_features_train = np.zeros((X_train_selected.shape[0], len(base_models)))
meta_features_test = np.zeros((X_test_selected.shape[0], len(base_models)))
meta_features_val = np.zeros((validation_X_selected.shape[0], len(base_models)))

meta_features_train_qda = np.zeros((X_train_qda_selected.shape[0], 1))
meta_features_test_qda = np.zeros((X_test_qda_selected.shape[0], 1))
meta_features_val_qda = np.zeros((validation_X_qda_selected.shape[0], 1))

# Train base models and get predictions
for i, (name, model) in enumerate(base_models):
    if name == 'qda':
        model.fit(X_train_qda_selected, y_train_qda)
        meta_features_train_qda[:, 0] = model.predict_proba(X_train_qda_selected)[:, 1]
        meta_features_test_qda[:, 0] = model.predict_proba(X_test_qda_selected)[:, 1]
        meta_features_val_qda[:, 0] = model.predict_proba(validation_X_qda_selected)[:, 1]
    else:
        model.fit(X_train_selected, y_train)
        meta_features_train[:, i] = model.predict_proba(X_train_selected)[:, 1]
        meta_features_test[:, i] = model.predict_proba(X_test_selected)[:, 1]
        meta_features_val[:, i] = model.predict_proba(validation_X_selected)[:, 1]

# Combine QDA meta-features with other base models
meta_features_train_combined = np.hstack([meta_features_train, meta_features_train_qda])
meta_features_test_combined = np.hstack([meta_features_test, meta_features_test_qda])
meta_features_val_combined = np.hstack([meta_features_val, meta_features_val_qda])

# Meta-model
meta_model = XGBClassifier()
meta_model.fit(meta_features_train_combined, y_train)

# Predict using the meta-model
meta_model_predictions_train = meta_model.predict(meta_features_train_combined)
meta_model_predictions_test = meta_model.predict(meta_features_test_combined)
meta_model_predictions_val = meta_model.predict(meta_features_val_combined)

# Calculate performance metrics
def print_metrics(name, y_true, predictions):
    accuracy = accuracy_score(y_true, predictions)
    mcc = matthews_corrcoef(y_true, predictions)
    f1 = f1_score(y_true, predictions)
    cm = confusion_matrix(y_true, predictions)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1]) if (cm[0, 0] + cm[0, 1]) > 0 else 0
    print(f"{name} - Accuracy:", accuracy)
    print(f"{name} - MCC:", mcc)
    print(f"{name} - F1 Score:", f1)
    print(f"{name} - Specificity:", specificity)

print_metrics("Train Data", y_train, meta_model_predictions_train)
print_metrics("Test Data", y_test, meta_model_predictions_test)
print_metrics("Validation Data", validation_y, meta_model_predictions_val)

<ipython-input-2-6878be04c381>:101: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('ann', KerasClassifier(build_fn=create_ann_model, input_dim=X_train_selected.shape[1])),
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


27/27 [==============================] - 0s 3ms/step
Train Data - Accuracy: 0.9985507246376811
Train Data - MCC: 0.9971056379236809
Train Data - F1 Score: 0.9985528219971056
Train Data - Specificity: 0.9971014492753624
Test Data - Accuracy: 0.958092485549133
Test Data - MCC: 0.9161887976156524
Test Data - F1 Score: 0.9580318379160636
Test Data - Specificity: 0.9595375722543352
Validation Data - Accuracy: 0.9699074074074074
Validation Data - MCC: 0.9398551043917015
Validation Data - F1 Score: 0.9697674418604652
Validation Data - Specificity: 0.9745370370370371


Saving base and meta models

In [3]:
import pickle

# Save the meta-model
with open('meta_model.pkl', 'wb') as f:
    pickle.dump(meta_model, f)

# Save the base models
for name, model in base_models:
    with open(f'{name}_model.pkl', 'wb') as f:
        pickle.dump(model, f)